In [10]:
import pandas as pd
import numpy as np
import requests as rq
import time
from requests.exceptions import HTTPError
import json
from jsonschema import validate

In [11]:
schema_summoner = {
    
    "id": {"type": "string"},
    "accountId": {"type": "int"},
    "puuid": {"type": "long"},
    "name": {"type": "string"},
    "profileIconId": {"type": "string"},
    "revisionDate": {"type": "string"},
    "summonerLevel": {"type": "long"}  
}

In [12]:
API_KEY = "RGAPI-1efd7d52-9210-470d-9595-ac6a2d38ae44"
base_diamond = "https://br1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/"
base_challenger = "https://br1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5/"
base_master = "https://br1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5"
base_grand_master = "https://br1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5"

base_summoner_infos = "https://br1.api.riotgames.com/lol/summoner/v4/summoners/by-name/"

In [13]:
def makeRequest(string):
    try:
        response = rq.get(string)
        response.raise_for_status()
    except HTTPError:
        if response.status_code == 429:
            time.sleep(120)
            makeRequest(string)
    return response.json()

In [14]:
def getLowEloInfos(json):
    df = pd.DataFrame(json)
    df["tier"] = df['tier'] + df["rank"]
    df.drop(["leagueId","queueType","rank","miniSeries"], axis=1,inplace=True)
    return df   

In [15]:
def getHighEloInfos(json):
    df_aux = pd.DataFrame(json["entries"])
    df_aux.drop("rank", axis=1, inplace=True)
    df_aux["tier"] = "MASTER"
    df_aux = df_aux.reindex(
        columns=['tier', 'summonerId', 'summonerName', 
                 'leaguePoints', 'wins', 'losses','veteran',
                 'inactive', 'freshBlood', 'hotStreak'])
    return df_aux

In [16]:
%%time
divisions = ["I","II","III","IV"]
tiers = ["DIAMOND","MASTER","GRANDMASTER","CHALLENGER"]
df = pd.DataFrame()
for i in tiers:
    if i == "DIAMOND":
        for j in divisions:
            string = base_diamond + i + "/" + j + "?api_key=" + API_KEY
            df_aux = getLowEloInfos(makeRequest(string))
            df = df.append(df_aux,ignore_index = True)
    
    elif i == "MASTER":
        string = base_master + "?api_key=" + API_KEY
        df_aux = getHighEloInfos(makeRequest(string))
        df = df.append(df_aux,ignore_index = True)
    
    
    elif i == "GRANDMASTER":
        string = base_grand_master + "?api_key=" + API_KEY
        df_aux = getHighEloInfos(makeRequest(string))
        df = df.append(df_aux,ignore_index = True)
    
    else:
        string = base_challenger + "?api_key=" + API_KEY
        df_aux = getHighEloInfos(makeRequest(string))
        df = df.append(df_aux,ignore_index = True)
        

CPU times: user 278 ms, sys: 33.5 ms, total: 312 ms
Wall time: 6.24 s


In [17]:
df.head()

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,DIAMONDI,a6XNnr-OC1673HAJaeCorjI1T9Sbe7gYJIBpbsu_CllJIOI,LokiFc,100,188,177,False,False,False,False
1,DIAMONDI,6YbR4YdY-AewAr2ihQtT_qHhJyNGtHUZNDLexZBQGU7ZZGQ,Janna Jana Janna,6,291,291,False,False,False,False
2,DIAMONDI,h_TlijF-s3RAtZaCyUVvBvgO-GGVuG60TqH3h7cTGrOKT18,lepanta,75,120,100,False,False,False,False
3,DIAMONDI,8fb8V6VEtBGfFZgq3Fyv3eEJ43UQyIeFOCm0SKEQvgu673M,i still hate you,0,350,298,False,False,False,False
4,DIAMONDI,yMjxJWZ9bgYm-QRQTLetKFsSDxM04GaLxNSdQSOpP_NsBW0,catch a fireee,0,423,389,True,False,False,False


In [18]:
df.to_csv("usersBasicInfos.csv",index=False)

In [19]:
df = pd.read_csv("usersBasicInfos.csv")

In [20]:
df.shape

(2681, 10)

In [21]:
df.head()

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,DIAMONDI,a6XNnr-OC1673HAJaeCorjI1T9Sbe7gYJIBpbsu_CllJIOI,LokiFc,100,188,177,False,False,False,False
1,DIAMONDI,6YbR4YdY-AewAr2ihQtT_qHhJyNGtHUZNDLexZBQGU7ZZGQ,Janna Jana Janna,6,291,291,False,False,False,False
2,DIAMONDI,h_TlijF-s3RAtZaCyUVvBvgO-GGVuG60TqH3h7cTGrOKT18,lepanta,75,120,100,False,False,False,False
3,DIAMONDI,8fb8V6VEtBGfFZgq3Fyv3eEJ43UQyIeFOCm0SKEQvgu673M,i still hate you,0,350,298,False,False,False,False
4,DIAMONDI,yMjxJWZ9bgYm-QRQTLetKFsSDxM04GaLxNSdQSOpP_NsBW0,catch a fireee,0,423,389,True,False,False,False


#### Agora vamos enriquecer esses dados com mais informações 

In [22]:
def checkFields(response, fields):
    for i in fields:
        if i not in response:
            return False
    return True

In [52]:
def getSummonerInfos(summonerNames):
    list_dicts = []
    count = 0
    for i in summonerNames:
        count+=1
        if (count % 100) == 0:
            df_temp = pd.DataFrame(list_dicts)
            df_temp.to_csv("DataSummornerTemp.csv")
        string = base_summoner_infos + i + "?api_key=" + API_KEY
        response = makeRequest(string)
        infos_dict = {}
        if checkFields(response,["accountId","summonerLevel","puuid"]):
            infos_dict["summonerName"] = i
            infos_dict["accountId"] = response["accountId"]
            infos_dict["summonerLevel"] = response["summonerLevel"]
            infos_dict["puuid"] = response["puuid"]
            list_dicts.append(infos_dict)
    return list_dicts

In [40]:
names = df.summonerName.to_list()

In [31]:
len(names)

2681

In [49]:
df_backup = pd.read_csv("DataSummornerTemp.csv")

In [53]:
%%time
infos_plus = getSummonerInfos(names)

CPU times: user 1min 27s, sys: 7.83 s, total: 1min 35s
Wall time: 1h 24min 35s


In [55]:
len(infos_plus)

2644

In [56]:
df_summoner = pd.DataFrame(infos_plus)

In [58]:
df_summoner.head()

,summonerName,accountId,summonerLevel,puuid
0,LokiFc,bk1m2_GS-2L9loYOKil6EbsEHnQOCY15LRVByLuQvjf46Cc,289,AvAT9pAFILqL0Eo0kWmFOXug4KyIBh4QprnfHU4Ao2e3Bj...
1,Janna Jana Janna,0-0irM82Yyb9ls9AQ2PGuGHjqn2pWBFVAuKEjM4qXeu5zHk,329,xo9H9-nK8p0qks6P39ceIIluPBqAGujYR1YZSC_JqWZxYD...
2,lepanta,B6nuFHgma2xw3B4ZYsSQeb38pgQxgajWykeB8OEZHm0sSO8,130,z2NP0f9IPVNevXHBEiZ5wjOHHwmqifJM0St3OpTjgI8xon...
3,i still hate you,4FnPhlgfii3JW283EbPr2Zl4ZPYoVdk6Qfjc7LIPP8MrZEI,224,CZq0S_gurM-JdaLAI8mZxCHhvxWGSen9ipXUt6QCOkyNDz...
4,catch a fireee,p5gtfCphMzSZUHv_hGKI4gAdej_ezQcgvHelm-rvB_878ec,131,EVZfgcJKxbwTCbnvj72it5d9gBeI52DNAoOL7dDKbWgwDp...


In [59]:
df_summoner.to_csv("summonerInfosPlus.csv",index=False)

In [60]:
df_summoner = pd.read_csv("summonerInfosPlus.csv")